In [19]:
import cv2
import os
from PIL import Image
import numpy as np

image_directory='Datasets/'

no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]

print(len(no_tumor_images))

1500


In [20]:
INPUT_SIZE=64

for i , image_name in enumerate(no_tumor_images):
    image=cv2.imread(image_directory+'no/'+image_name)
    image=Image.fromarray(image,'RGB')
    image=image.resize((INPUT_SIZE,INPUT_SIZE))
    dataset.append(np.array(image))
    label.append(0)

for i , image_name in enumerate(yes_tumor_images):
    image=cv2.imread(image_directory+'yes/'+image_name)
    image=Image.fromarray(image, 'RGB')
    image=image.resize((INPUT_SIZE,INPUT_SIZE))
    dataset.append(np.array(image))
    label.append(1)

print(len(dataset))
print(len(label))

3000
3000


In [21]:
dataset=np.array(dataset)
label=np.array(label)

In [25]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.2, random_state=0)

print(x_train.shape)
print(x_test.shape)

(2400, 64, 64, 3)
(600, 64, 64, 3)


In [26]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize

x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

2023-10-23 13:32:59.995983: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 13:33:00.022276: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 13:33:00.022316: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 13:33:00.022335: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 13:33:00.027210: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 13:33:00.028145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model=Sequential()

model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5)) # to stop from overfiting
model.add(Dense(1))
model.add(Activation('sigmoid'))

2023-10-23 13:38:03.964315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-23 13:38:03.964933: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [28]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [29]:
model.fit(x_train, y_train, 
batch_size=32, 
verbose=1, epochs=10, 
validation_data=(x_test, y_test),
shuffle=False)

Epoch 1/10


2023-10-23 13:41:25.236475: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 117964800 exceeds 10% of free system memory.


 3/75 [>.............................] - ETA: 4s - loss: 0.6969 - accuracy: 0.5312

2023-10-23 13:41:26.048725: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28096128 exceeds 10% of free system memory.
2023-10-23 13:41:26.048823: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28096128 exceeds 10% of free system memory.
2023-10-23 13:41:26.109941: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28096128 exceeds 10% of free system memory.
2023-10-23 13:41:26.110086: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28096128 exceeds 10% of free system memory.


75/75 [==============================] - 6s 65ms/step - loss: 0.5651 - accuracy: 0.7092 - val_loss: 0.5495 - val_accuracy: 0.7250
Epoch 2/10
75/75 [==============================] - 5s 62ms/step - loss: 0.4322 - accuracy: 0.8146 - val_loss: 0.4544 - val_accuracy: 0.8033
Epoch 3/10
75/75 [==============================] - 4s 60ms/step - loss: 0.3627 - accuracy: 0.8492 - val_loss: 0.3598 - val_accuracy: 0.8383
Epoch 4/10
75/75 [==============================] - 4s 54ms/step - loss: 0.2851 - accuracy: 0.8858 - val_loss: 0.2795 - val_accuracy: 0.8683
Epoch 5/10
75/75 [==============================] - 4s 59ms/step - loss: 0.2231 - accuracy: 0.9146 - val_loss: 0.2224 - val_accuracy: 0.9133
Epoch 6/10
75/75 [==============================] - 5s 66ms/step - loss: 0.1796 - accuracy: 0.9358 - val_loss: 0.1686 - val_accuracy: 0.9417
Epoch 7/10
75/75 [==============================] - 5s 64ms/step - loss: 0.1249 - accuracy: 0.9596 - val_loss: 0.1467 - val_accuracy: 0.9550
Epoch 8/10
75/75 [======

In [30]:
model.save('BrainTumorDSAI.h5')

/home/suraj/Documents/DSAI-submission/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
